In [ ]:
import cv2
import os
import time
import json
import random
from PIL import Image
import numpy as np
from mss import mss
import serial, time
import binascii
import imageio
import urllib
import requests

In [ ]:
cap = cv2.VideoCapture(0)
sct = mss()

In [57]:
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)
src_dev = 0
w, h = 64, 64
monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}

In [55]:
gifurl = 'https://gifs.iloopit.net/resources/1ead57a3-48e4-4fb1-86ea-07527a7c079a/converted.gif'
fname = 'tmp.gif'

## Read the gif from the web, save to the disk
if not os.path.exists(fname):
    imdata = urllib.request.urlopen(gifurl).read()
    imbytes = bytearray(imdata)
    open(fname,"wb+").write(imdata)

#gif = imageio.mimread('/Users/dusanklinec/Downloads/511e39e7152f0.gif') #fname
gif = imageio.mimread(fname) #
num_frames = len(gif)
print("Total {} frames in the gif!".format(num_frames))

imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in gif]
imgs = [cv2.resize(rgb, (64, 64)) for rgb in imgs]
imgs = [cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1] for img in imgs]

Total 108 frames in the gif!


In [23]:
# imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in gif]
# imgs = [cv2.resize(rgb, (64, 64)) for rgb in imgs]
# imgs = [cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1] for img in imgs]
## Display the gif
i = 0

while True:
    cv2.imshow("gif", imgs[i])
    if cv2.waitKey(100)&0xFF == 27:
        break
    i = (i+1)%nums
cv2.destroyAllWindows()

IndexError: list index out of range

In [67]:
cframe = 0
while True:
    while True:
        rline = arduino.readline().strip()
        if len(rline) > 0:
            print(rline)
        else:
            break
        
    if src_dev == 0:
        ret, frame = cap.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif src_dev == 1:
        rgb = np.array(sct.grab(monitor))
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    elif src_dev == 2:
        rgb = imgs[cframe]
        cframe = (cframe + 15) % num_frames
    else:
        raise ValueError('wtf')
    obuff = rgb
    
    #obuff = cv2.adaptiveThreshold(obuff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 1)
    obuff = cv2.resize(obuff, (64, 64))#, interpolation = cv2.INTER_NEAREST)
    #ret, obuff = cv2.threshold(obuff, 128, 255, cv2.THRESH_BINARY)
    obuff = cv2.adaptiveThreshold(obuff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 1)
    
    nobuff = bytearray(int(64*64/8))  # bitmap picture, 1 Byte = 8 pixels
    cbit = 0
    for i in range(64*64):
        bout = obuff[i//64][i%64] > 0
        if bout:
            nobuff[i//8] |= 1 << (i%8)
            
    #print(binascii.hexlify(nobuff))

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64

        ln = arduino.write(sbuff)
        arduino.flush()
        #print('%s, %s' % (ln, binascii.hexlify(sbuff)))
        time.sleep(0.15)
   
    cv2.imshow('frame', obuff)
    cv2.waitKey(1)
    #time.sleep(2)
    #break


KeyboardInterrupt: 

In [ ]:
cv2.destroyAllWindows()

In [ ]:
#w, h = 200, 200
#monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
#img = Image.frombytes('RGB', (w,h), sct.grab(monitor).rgb)
#cv2.imshow('frame', np.array(img))
cv2.imshow('frame', np.array(sct.grab(monitor)))
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
    

In [ ]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

In [ ]:
cap.release()

In [ ]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


In [ ]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [ ]:
while True:
    print(arduino.readline())